In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bs4 import BeautifulSoup as bs
import json
import jsonlines
from tqdm import tqdm

In [ ]:
file = open('/content/drive/MyDrive/graduate/nlp/week5/med.cd.az.xml', 'r', encoding = 'utf-8')
content = file.readlines()
bs_contents = []
for line in tqdm(content):
    bs_contents.append(bs(line, 'xml'))
print(len(bs_contents))

100%|██████████| 44309/44309 [00:31<00:00, 1419.19it/s]

44309


In [ ]:
import re

word_dict = {}
for bs_content in tqdm(bs_contents):
  for test in bs_content.find_all('HOMOGRAPH'):
    if test.find('HEAD') != None:
      if test.find(re.compile('ENTRY*')) != None:
                  word = test.find(re.compile('ENTRY*')).text
                  word_dict[word] = {} 
      if test.find(re.compile('ENTRY13')) != None:  
        word = test.find(re.compile('ENTRY13')).text
        word_dict[word] = {}        
      if test.attrs['ID'] != None:
        ID = test.attrs['ID']
        word_dict[word].update({"ID": ID})
      if test.find('PART-OF-SPEECH') != None:
        PART_OF_SPEECH = test.find('PART-OF-SPEECH').text
        word_dict[word].update({"PART-OF-SPEECH": PART_OF_SPEECH})
      if test.find('STYLE-LEVEL') != None:
        STYLE_LEVEL = test.find('STYLE-LEVEL').text
        word_dict[word].update({"STYLE-LEVEL": STYLE_LEVEL})

    if test.find('PHRASE') != None:
      phrase_dic= {}
      for phrase_entry in test.find_all('PHRASE'):
        if phrase_entry.find('DEFINITION'):
          def_tag = phrase_entry.find('DEFINITION').name
          def_text = phrase_entry.find('DEFINITION').text
          phrase_dic[def_tag] = def_text
        if phrase_entry.find('MULTIWORD'):
          multiword_tag = phrase_entry.find('MULTIWORD').name
          multiword_text = phrase_entry.find('MULTIWORD').text
          phrase_dic[multiword_tag] = multiword_text
        if phrase_entry.find('TRANSLATION'):
          translation_tag = phrase_entry.find('TRANSLATION').name
          translation_text = phrase_entry.find('TRANSLATION').text
          phrase_dic[translation_tag] = translation_text
        if phrase_entry.find('trans'):
          trans_tag = phrase_entry.find('trans').name
          trans_text = phrase_entry.find('trans').text
          phrase_dic[trans_tag] = trans_text
      word_dict[word].update({"PHRASE": [phrase_dic]})


    if test.find(re.compile('SENSE*')) != None:
      sense_dic= {}
      for sense_entry in test.find_all(re.compile('SENSE*')):
        if sense_entry.find('DEFINITION'):
          def_tag = sense_entry.find('DEFINITION').name
          def_text = sense_entry.find('DEFINITION').text
          sense_dic[def_tag] = def_text
        if sense_entry.find("trans"):
          trans_tag = sense_entry.find("trans").name
          trans_text = sense_entry.find("trans").text
          sense_dic[trans_tag] = trans_text
        if sense_entry.find("TRANSLATION"):
          translation_tag = sense_entry.find("TRANSLATION").name
          translation_text = sense_entry.find("TRANSLATION").text
          sense_dic[translation_tag] = translation_text
        if sense_entry.find("EXAMPLES"):
          examples_tag = sense_entry.find("EXAMPLES").name
          examples_list = []
          for example_entry in sense_entry.find_all("EXAMPLES"):
            example_dic = {}
            if example_entry.find('EXAMPLE'):
              example_tag = example_entry.find('EXAMPLE').name
              example_text = example_entry.find('EXAMPLE').text
              example_dic[example_tag] = example_text
            if example_entry.find('TRANSLATION'):
              example_TRANSLATION_tag = example_entry.find('TRANSLATION').name
              example_TRANSLATION_text = example_entry.find('TRANSLATION').text
              example_dic.update({example_TRANSLATION_tag:example_TRANSLATION_text})
            if example_entry.find('trans'):
              example_trans_tag = example_entry.find('trans').name
              example_trans_text = example_entry.find('trans').text
              example_dic.update({example_trans_tag:example_trans_text})
            examples_list.append(example_dic)
          sense_dic[examples_tag] = examples_list
        if sense_entry.find("SUB-SENSE"):
          sub_sense_tag = sense_entry.find("SUB-SENSE").name
          sub_sense_list = []
          for sub_sense_entry in sense_entry.find_all("SUB-SENSE"):
            sub_sense_dic = {}
            if sub_sense_entry.find('DEFINITION'):
              sub_sense_def_tag = sub_sense_entry.find('DEFINITION').name
              sub_sense_def_text = sub_sense_entry.find('DEFINITION').text
              sub_sense_dic[sub_sense_def_tag] = sub_sense_def_text
            if sub_sense_entry.find("TRANSLATION"):
              sub_sense_TRANSLATION_tag = sub_sense_entry.find('TRANSLATION').name
              sub_sense_TRANSLATION_text = sub_sense_entry.find('TRANSLATION').text
              sub_sense_dic.update({sub_sense_TRANSLATION_tag:sub_sense_TRANSLATION_text})
            if sub_sense_entry.find('trans'):
              sub_sense_trans_tag = sub_sense_entry.find('trans').name
              sub_sense_trans_text = sub_sense_entry.find('trans').text
              sub_sense_dic.update({sub_sense_trans_tag:sub_sense_trans_text})
            if sub_sense_entry.find("EXAMPLES"):
              sub_sense_examples_tag = sub_sense_entry.find("EXAMPLES").name
              sub_sense_examples_list = []
              for example_entry in sub_sense_entry.find_all("EXAMPLES"):
                example_dic = {}
              if example_entry.find('EXAMPLE'):
                example_tag = example_entry.find('EXAMPLE').name
                example_text = example_entry.find('EXAMPLE').text
                example_dic[example_tag] = example_text
              if example_entry.find('TRANSLATION'):
                example_TRANSLATION_tag = example_entry.find('TRANSLATION').name
                example_TRANSLATION_text = example_entry.find('TRANSLATION').text
                example_dic.update({example_TRANSLATION_tag:example_TRANSLATION_text})
              if example_entry.find('trans'):
                example_trans_tag = example_entry.find('trans').name
                example_trans_text = example_entry.find('trans').text
                example_dic.update({example_trans_tag:example_trans_text})
              sub_sense_examples_list.append(example_dic)
              sub_sense_dic.update({sub_sense_examples_tag:sub_sense_examples_list})
            sub_sense_list.append(sub_sense_dic)
          sense_dic[sub_sense_tag] = sub_sense_list
      word_dict[word].update({"SENSE": [sense_dic]})
   
    if test.find('SENSE45') != None:
      sense_dic= {}
      for sense_entry in test.find_all('SENSE45'):
        if sense_entry.find('DEFINITION'):
          def_tag = sense_entry.find('DEFINITION').name
          def_text = sense_entry.find('DEFINITION').text
          sense_dic[def_tag] = def_text
        if sense_entry.find("trans"):
          trans_tag = sense_entry.find("trans").name
          trans_text = sense_entry.find("trans").text
          sense_dic[trans_tag] = trans_text
        if sense_entry.find("TRANSLATION"):
          translation_tag = sense_entry.find("TRANSLATION").name
          translation_text = sense_entry.find("TRANSLATION").text
          sense_dic[translation_tag] = translation_text
        if sense_entry.find("EXAMPLES"):
          examples_tag = sense_entry.find("EXAMPLES").name
          examples_list = []
          for example_entry in sense_entry.find_all("EXAMPLES"):
            example_dic = {}
            if example_entry.find('EXAMPLE'):
              example_tag = example_entry.find('EXAMPLE').name
              example_text = example_entry.find('EXAMPLE').text
              example_dic[example_tag] = example_text
            if example_entry.find('TRANSLATION'):
              example_TRANSLATION_tag = example_entry.find('TRANSLATION').name
              example_TRANSLATION_text = example_entry.find('TRANSLATION').text
              example_dic.update({example_TRANSLATION_tag:example_TRANSLATION_text})
            if example_entry.find('trans'):
              example_trans_tag = example_entry.find('trans').name
              example_trans_text = example_entry.find('trans').text
              example_dic.update({example_trans_tag:example_trans_text})
            examples_list.append(example_dic)
          sense_dic[examples_tag] = examples_list
        if sense_entry.find("SUB-SENSE"):
          sub_sense_tag = sense_entry.find("SUB-SENSE").name
          sub_sense_list = []
          for sub_sense_entry in sense_entry.find_all("SUB-SENSE"):
            sub_sense_dic = {}
            if sub_sense_entry.find('DEFINITION'):
              sub_sense_def_tag = sub_sense_entry.find('DEFINITION').name
              sub_sense_def_text = sub_sense_entry.find('DEFINITION').text
              sub_sense_dic[sub_sense_def_tag] = sub_sense_def_text
            if sub_sense_entry.find("TRANSLATION"):
              sub_sense_TRANSLATION_tag = sub_sense_entry.find('TRANSLATION').name
              sub_sense_TRANSLATION_text = sub_sense_entry.find('TRANSLATION').text
              sub_sense_dic.update({sub_sense_TRANSLATION_tag:sub_sense_TRANSLATION_text})
            if sub_sense_entry.find('trans'):
              sub_sense_trans_tag = sub_sense_entry.find('trans').name
              sub_sense_trans_text = sub_sense_entry.find('trans').text
              sub_sense_dic.update({sub_sense_trans_tag:sub_sense_trans_text})
            if sub_sense_entry.find("EXAMPLES"):
              sub_sense_examples_tag = sub_sense_entry.find("EXAMPLES").name
              sub_sense_examples_list = []
              for example_entry in sub_sense_entry.find_all("EXAMPLES"):
                example_dic = {}
              if example_entry.find('EXAMPLE'):
                example_tag = example_entry.find('EXAMPLE').name
                example_text = example_entry.find('EXAMPLE').text
                example_dic[example_tag] = example_text
              if example_entry.find('TRANSLATION'):
                example_TRANSLATION_tag = example_entry.find('TRANSLATION').name
                example_TRANSLATION_text = example_entry.find('TRANSLATION').text
                example_dic.update({example_TRANSLATION_tag:example_TRANSLATION_text})
              if example_entry.find('trans'):
                example_trans_tag = example_entry.find('trans').name
                example_trans_text = example_entry.find('trans').text
                example_dic.update({example_trans_tag:example_trans_text})
              sub_sense_examples_list.append(example_dic)
              sub_sense_dic.update({sub_sense_examples_tag:sub_sense_examples_list})
            sub_sense_list.append(sub_sense_dic)
          sense_dic[sub_sense_tag] = sub_sense_list
      word_dict[word].update({"SENSE45": [sense_dic]})


100%|██████████| 44309/44309 [01:08<00:00, 644.47it/s]


In [ ]:
word_dict['fall1']
# agony --> 加入 'STYLE-LEVEL'
# SUB_SENSE 底下 Examples 印出

{'ID': '124631',
 'PART-OF-SPEECH': 'verb',
 'STYLE-LEVEL': 'mainly literary',
 'PHRASE': [{'DEFINITION': 'if you fall into step beside someone, you start walking beside them',
   'MULTIWORD': 'fall into step',
   'trans': '開始（與…）並肩而行'}],
 'SENSE': [{'DEFINITION': 'if a particular job or duty falls to someone, it is their responsibility',
   'trans': '由…負責',
   'EXAMPLES': [{'EXAMPLE': 'It fell to me to explain to him what happened.',
     'trans': '由我負責向他\ue468釋所發生的事。'}],
   'SUB-SENSE': [{'DEFINITION': 'to do a particular job or activity in a careless way so that you are not successful',
     'trans': '（因為馬虎而）失敗'}]}],
 'SENSE45': [{'DEFINITION': 'if the emphasis falls on a particular part of a word, you emphasize that part when you say or sing it',
   'trans': '（重音或強調部分）落在…上',
   'EXAMPLES': [{'EXAMPLE': 'In English, the stress in Paris falls on the first syllable.',
     'trans': '在英語中，Paris 的重音落在第一個音節。'}],
   'SUB-SENSE': [{'DEFINITION': 'if a place falls in a war, a different army